# **1. 가상환경 설정 및 라이브러리 불러오기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
!pip install ptflops
from ptflops import get_model_complexity_info

try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

  Preparing metadata (setup.py) ... done
  Created wheel for ptflops: filename=ptflops-0.7.1.2-py3-none-any.whl size=13214 sha256=44509af9f8a4070d14548a008c943693bd8ad84c4c8d8a03b19fd140ead6364a
  Stored in directory: /root/.cache/pip/wheels/9d/90/07/20e8c3221349a85d63b319593e1bcbb6e0c995d2e2bcc5d775
Successfully built ptflops


In [ ]:
import random
# device : gpu를 사용할 경우에는 'cuda', 그렇지 않을 경우에는 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 랜덤 시드 고정
# 실험 조건을 동일하게 설정하여 같은 input을 넣으면 같은 결과가 나올 수 있도록 함
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
# load the data
train_transform = transforms.Compose(
    [
     transforms.RandomCrop(32, padding=4),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784))])

test_transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784))])

ds = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)


test_ds = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)

# split the training set and validation set
torch.manual_seed(50)
test_size = len(test_ds)
val_size = 5000
train_size = len(ds) - val_size
batch_size = 256

train_ds, val_ds = random_split(ds, [train_size, val_size])


train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)

val_loader = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4)

test_loader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4)

classes = ds.classes

print(train_size)
print(val_size)
print(test_size)
print(torch.cuda.is_available())

100%|██████████| 170498071/170498071 [00:03<00:00, 49571651.33it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
45000
5000
10000
True


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
### original model ###

__all__ = ['ResNet', 'resnet18']

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError(
                'BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError(
                "Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1,
                            bias=False)

        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

# Teacher model build
model_teacher = ResNet(block=BasicBlock, layers=[2, 2, 2, 2], num_classes=10)

# Teacher model checkpoint 적용
model_teacher.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/checkpoints/original_best.pth'))

<All keys matched successfully>

In [ ]:
### lightweight model ###

__all__ = ['ResNet', 'resnet18']


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=in_planes, bias=False, dilation=dilation)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock_light(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock_light, self).__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError(
                'BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError(
                "Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.dwconv33_1 = conv3x3(inplanes, planes, stride)
        self.conv11_1 = conv1x1(planes, planes)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)

        self.dwconv33_2 = conv3x3(planes, planes)
        self.conv11_2 = conv1x1(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.dwconv33_1(x)
        out = self.conv11_1(out)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.dwconv33_2(out)
        out = self.conv11_2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class Bottleneck_light(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck_light, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet_lightweight(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet_lightweight, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1,
                            bias=False)

        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck_light):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock_light):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

# Student model build
model_student = ResNet_lightweight(block=BasicBlock_light, layers=[2, 2, 2, 2], num_classes=10)

In [ ]:
import time

def validate(model, criterion, val_loader, use_gpu=False):
  val_size = len(val_loader.dataset)
  val_loss = 0
  correct = 0
  device = torch.device( "cuda:0" if use_gpu else "cpu" )

  with torch.no_grad():
    for i, data in enumerate(val_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # forward + backward + optimize
        outputs = model(inputs).to(device)
        loss = criterion(outputs, labels)

        val_loss += loss * inputs.size(0)

        # val accuracy
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

    val_loss = val_loss/val_size
    val_accuracy = correct/val_size;

  return val_loss, val_accuracy

def get_train_accuracy(model, criterion, train_loader, use_gpu=False):
   train_size = len(train_loader.dataset)
   correct = 0
   device = torch.device( "cuda:0" if use_gpu else "cpu" )

   with torch.no_grad():
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # forward + backward + optimize
        outputs = model(inputs).to(device)

        # val accuracy
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

    print(correct)
    return correct/train_size

def train(model_student,model_teacher, criterion, criterion_kd, optimizer, train_loader, val_loader, epoch=2, use_gpu=False):
  best_loss = 999999
  train_size = len(train_loader.dataset)
  device = torch.device( "cuda:0" if use_gpu else "cpu" )

  history = {
      'train_loss': [],
      'train_accuracy': [],
      'val_loss': [],
      'val_accuracy': []
      }

  model_student = model_student.to(device)
  model_teacher = model_teacher.to(device)

  for epoch in range(epoch):  # loop over the dataset multiple times
    running_loss = 0.0
    correct = 0

    print(f'------------------------------\n Epoch: {epoch + 1}')

    t1 = time.time()
    for  i,data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs_student = model_student(inputs)
        outputs_teacher = model_teacher(inputs)

        loss_label = criterion(outputs_student, labels)
        loss_kd = criterion_kd(outputs_student,outputs_teacher)


        loss = loss_label + loss_kd

        loss.backward()
        optimizer.step()

        # as loss.item() return the average batch loss, so convert it to the total loss
        running_loss += loss.item() * inputs.size(0)

    t2 = time.time()
    t = t2 - t1

    # save the model
    # PATH = f'./drive/My Drive/Colab Notebooks/checkpoints/resnet18-original-e{epoch}.pth'
    PATH = './drive/My Drive/Colab Notebooks/checkpoints/'
    # if epoch % 5 == 0:
    #   torch.save({'epoch': epoch, 'state_dict': model_student.state_dict(), 'optimizer': optimizer.state_dict(),}, PATH)

    if loss < best_loss:
      best_loss = loss
      torch.save(model_student.state_dict(), os.path.join(PATH, "Knowledge_distillation_student_best.pth"))

    epoch_train_loss = running_loss/train_size
    epoch_train_accuracy = get_train_accuracy(model_student, criterion, train_loader, use_gpu)
    epoch_val_loss, epoch_val_accuracy = validate(model_student, criterion, val_loader, use_gpu)
    print(f'time: {int(t)}sec train_loss: {epoch_train_loss}, train_accuracy: {epoch_train_accuracy}, val_loss: {epoch_val_loss}, val_accuracy: {epoch_val_accuracy}');

    history['train_loss'].append(epoch_train_loss)
    history['train_accuracy'].append(epoch_train_accuracy)
    history['val_loss'].append(epoch_val_loss)
    history['val_accuracy'].append(epoch_val_accuracy)

  return history

In [ ]:
optimizer = optim.SGD(model_student.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

criterion = nn.CrossEntropyLoss()

def CriterionKD(pred,soft, temperature = 4):
    '''
    knowledge distillation loss
    '''
    criterion_kd = torch.nn.KLDivLoss()
    scale_pred = pred
    scale_soft = soft
    loss = criterion_kd(F.log_softmax(scale_pred / temperature, dim=1), F.softmax(scale_soft / temperature, dim=1))

    return loss * temperature * temperature

In [ ]:
history = train(model_student,model_teacher, criterion, CriterionKD, optimizer, train_loader, val_loader, 10, torch.cuda.is_available())

------------------------------
 Epoch: 1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2943: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


26448
time: 28sec train_loss: 1.504368986807929, train_accuracy: 0.5877333333333333, val_loss: 1.1541022062301636, val_accuracy: 0.5842
------------------------------
 Epoch: 2
28890
time: 28sec train_loss: 1.275393845070733, train_accuracy: 0.642, val_loss: 1.0248682498931885, val_accuracy: 0.639
------------------------------
 Epoch: 3
29855
time: 28sec train_loss: 1.1575849747127958, train_accuracy: 0.6634444444444444, val_loss: 0.9633286595344543, val_accuracy: 0.6558
------------------------------
 Epoch: 4
31029
time: 28sec train_loss: 1.0486638008965385, train_accuracy: 0.6895333333333333, val_loss: 0.9066404104232788, val_accuracy: 0.6778
------------------------------
 Epoch: 5
31857
time: 26sec train_loss: 0.9744217832989163, train_accuracy: 0.7079333333333333, val_loss: 0.8675864934921265, val_accuracy: 0.6916
------------------------------
 Epoch: 6
32915
time: 27sec train_loss: 0.9003695823457506, train_accuracy: 0.7314444444444445, val_loss: 0.8165280222892761, val_accura

In [ ]:
def test(model, criterion, test_loader, use_gpu=False):
  test_size = len(test_loader.dataset)
  device = torch.device( "cuda:0" if use_gpu else "cpu" )
  test_loss = 0.0
  test_accuracy = 0
  correct = 0
  model.eval()
  with torch.no_grad():
      for i, data in enumerate(test_loader):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data[0].to(device), data[1].to(device)

          # forward + backward + optimize
          outputs = model(inputs).to(device)
          loss = criterion(outputs, labels)

          test_loss += loss * inputs.size(0)

          # val accuracy
          _, predicted = torch.max(outputs.data, 1)
          correct += (predicted == labels).sum().item()


      test_loss = test_loss/test_size
      test_accuracy = correct/test_size

  return test_loss, test_accuracy

In [ ]:
test_loss, test_accuracy = test(model_student, criterion, test_loader, True)
history['test_loss'] = test_loss
history['test_accuracy'] = test_accuracy

In [ ]:
print(history['test_loss'], history['test_accuracy'] )

tensor(0.6878, device='cuda:0') 0.7639


In [ ]:
net = model_student
macs, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
print(macs,'macs')
print(params,'params')

ResNet_lightweight(
  1.62 M, 100.000% Params, 21.89 MMac, 98.844% MACs, 
  (conv1): Conv2d(1.73 k, 0.107% Params, 1.77 MMac, 7.989% MACs, 3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, 0.008% Params, 131.07 KMac, 0.592% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(0, 0.000% Params, 65.54 KMac, 0.296% MACs, inplace=True)
  (maxpool): MaxPool2d(0, 0.000% Params, 65.54 KMac, 0.296% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    19.2 k, 1.188% Params, 4.98 MMac, 22.488% MACs, 
    (0): BasicBlock_light(
      9.6 k, 0.594% Params, 2.49 MMac, 11.244% MACs, 
      (dwconv33_1): Conv2d(576, 0.036% Params, 147.46 KMac, 0.666% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (conv11_1): Conv2d(4.1 k, 0.254% Params, 1.05 MMac, 4.734% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): B